In [1]:
import pandas as pd
import re


### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:  
- оценка 2 и меньше - низкий рейтинг  
- оценка 4 и меньше - средний рейтинг  
- оценка 4.5 и 5 - высокий рейтинг  

Результат классификации запишите в столбец class

In [2]:
ratings = pd.read_csv('ml-latest-small\\ratings.csv')


In [3]:
def rating_str(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    elif rating <= 4:
        return 'средний рейтинг'
    return 'высокий рейтинг'


In [4]:
ratings['class'] = ratings['rating'].apply(rating_str)
ratings.head(10)


,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
5,1,1263,2.0,1260759151,низкий рейтинг
6,1,1287,2.0,1260759187,низкий рейтинг
7,1,1293,2.0,1260759148,низкий рейтинг
8,1,1339,3.5,1260759125,средний рейтинг
9,1,1343,2.0,1260759131,низкий рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [5]:
data = pd.read_csv('ml-latest-small\\keywords.csv')
data[data['keyword'].str.contains('моск', case=False)]


,keyword,shows
127,авито москва,979292
143,эхо москвы,889657
155,школьный портал московской области,836987
197,погода в москве,745745
414,погода в москве на 14 дней,400914
...,...,...
99494,погода на 14 дней в москве,400914
99506,московское время сейчас точное,9740
99597,индексы москвы по адресу,3574
99810,услуги московской области школьный портал элек...,3541


In [6]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}


In [7]:
geo_data_inverted = {}
for region, cities in geo_data.items():
    for city in cities:
        geo_data_inverted[city] = region
geo_data_inverted


{'москва': 'Центр',
 'тула': 'Центр',
 'ярославль': 'Центр',
 'петербург': 'Северо-Запад',
 'псков': 'Северо-Запад',
 'мурманск': 'Северо-Запад',
 'владивосток': 'Дальний Восток',
 'сахалин': 'Дальний Восток',
 'хабаровск': 'Дальний Восток'}

In [8]:
def reg_class(keyword):
    for word in geo_data_inverted:
        if len(word) > 5 and len(re.findall(r'{}[а-я]'.format(word[:-2]), keyword)) > 0:
            return geo_data_inverted[word]
        elif len(word) <= 5 and len(re.findall(r'\b{}[а-я]'.format(word[:-1]), keyword)) > 0:
            return geo_data_inverted[word]
    return 'undefined'


In [9]:
data['region'] = data['keyword'].apply(reg_class)


In [10]:
data[data['keyword'].str.contains('сахал', case=False)].head(100)


,keyword,shows,region
21445,сахалин,14494,Дальний Восток
23547,южно сахалинск,13386,Дальний Восток
31701,сахалайф,10322,Дальний Восток
41988,сетевой город образование сахалинская область,7817,Дальний Восток
44291,сетевой город южно сахалинск,7515,Дальний Восток
58451,погода в южно сахалинске,6053,Дальний Восток
68736,красная икра сеть магазинов ооо сахалинская ры...,5069,Дальний Восток


### Задание 3

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    1. для каждой строки пройдите по всем годам списка years
    2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [11]:
ratings = pd.read_csv('ml-latest-small//ratings.csv')
movies = pd.read_csv('ml-latest-small//movies.csv')


In [12]:
years = list(range(1950, 2010))


In [13]:
movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
def production_year(title):
    for year in years:
        if len(re.findall(r'.+\s\({}\)'.format(year), title)) > 0:
            return year
    return 1900


In [15]:
movies['year'] = movies['title'].apply(production_year)


In [17]:
movies_ratings = pd.merge(movies, ratings)
movies_ratings.groupby('year').mean(['rating']).sort_values('rating', ascending=False)


,movieId,userId,rating,timestamp
year,,,,
1957,2750.965190,360.933544,4.014241,1.083707e+09
1972,3983.538976,359.694878,4.011136,1.122759e+09
1952,4107.796053,346.394737,4.000000,1.090512e+09
1954,2867.661850,358.228324,3.994220,1.070591e+09
1951,2605.588477,347.106996,3.983539,1.052714e+09
...,...,...,...,...
2005,37156.244600,348.509719,3.448434,1.273459e+09
2003,8154.379187,340.186204,3.444777,1.238031e+09
1996,909.740849,347.970249,3.426600,1.020020e+09
